In [1]:
from collections import defaultdict
from pprint import pprint

import gensim
import numpy as np
import pandas as pd
from surprise import Dataset, Reader
from tqdm import tqdm

from src.models import cf

tqdm.pandas()

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
CATEGORY = "Grocery_and_Gourmet_Food"

# training parameters
N_EPOCHS = 10
LR_ALL = 0.005
BETA = 0.1

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...
47769,77420,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A192LQZWDYPR4U,5.0,Another quality Reese Peanut Butter Cup produc...,2014-02-27,quality reese peanut butter cup product great ...
47770,77421,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A2QKXW3LDQ66P5,5.0,I purchased these for my husband who has every...,2013-02-20,purchase husband love reeses valentine day pre...
47771,77430,B00ID9VSOM,"Viva Labs Organic Coconut Sugar: Non-GMO, Low-...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2P3TGJU301KXD,5.0,this stuff is INCREDIBILY yummy! SO much bette...,2014-07-15,stuff incredibily yummy good regular brown sug...
47772,77456,B00IRL93SY,Barrie House Kenya Estate - AA Single Cup Caps...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AEFE9VDHTQ199,5.0,"Very nice aroma, body and taste! Will buy this...",2014-05-24,nice aroma body taste buy coffee good coffee a...
47773,77508,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2AEZQ3DGBBLPR,2.0,This is a no go for diabetics according to my ...,2014-06-26,diabetic accord wife doctor order intention us...


# Preparing Topic Vectors

In [4]:
class LDA:
    def __init__(self, reviews):
        self.reviews = reviews
        self.lda = None
        self.dictionary = None

    def train(self, n_topics=50, n_epochs=20, workers=8):
        # tokenizations
        dictionary = gensim.corpora.Dictionary(self.reviews)
        # filtering tokens less than 5 reviews, more than 0.85 reviews
        dictionary.filter_extremes(no_below=5, no_above=0.85)
        # creating dict how many words and time it appears
        bow_corpus = [dictionary.doc2bow(doc) for doc in self.reviews]
        
        # train model
        self.lda = gensim.models.LdaMulticore(bow_corpus, 
                                              num_topics=n_topics, 
                                              id2word=dictionary, 
                                              passes=n_epochs,
                                              workers=workers)
        # save dictionary
        self.dictionary = dictionary
        
    def get_document_topics(self, doc, minimum_probability=0.0):
        """
        """
        return self.lda.get_document_topics(doc, minimum_probability=minimum_probability)

In [5]:
# generating tokenized reviews
processed_reviews = train["processedReviewText"].progress_apply(lambda x: x.split())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47774/47774 [00:00<00:00, 171945.29it/s]


In [6]:
# instantiate lda model
lda_model = LDA(processed_reviews)

In [7]:
%%time
# training the LDA model
lda_model.train()

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similar

CPU times: user 2min 50s, sys: 18.3 s, total: 3min 8s
Wall time: 2min 57s


# Generating User/Item Topic Vectors

In [8]:
def get_topic_vectors(model, corpus, n_topics=50):
    """
    """
    topic_vecs = []
    for i in tqdm(range(len(corpus))):
        top_topics = model.get_document_topics(corpus[i])
        topic_vecs.append([top_topics[i][1] for i in range(n_topics)])
        
    return topic_vecs

def generate_user_item_vectors(lda: LDA, train: pd.DataFrame):
    """
    """
    user_reviews = train.groupby(["reviewerID"])['processedReviewText'].apply(lambda x: ' '.join(x))
    item_reviews = train.groupby(["asin"])["processedReviewText"].apply(lambda x: ' '.join(x))
    
    # get unique users and items
    unique_users = user_reviews.index.tolist()
    unique_items = item_reviews.index.tolist()
    
    # tokenize reviews
    user_reviews_list = user_reviews.apply(lambda x: x.split()).tolist()
    item_reviews_list = item_reviews.apply(lambda x: x.split()).tolist()
    
    # generate corpus based on aggregate of user/item reviews
    user_corpus = [lda.dictionary.doc2bow(doc) for doc in user_reviews_list]
    item_corpus = [lda.dictionary.doc2bow(doc) for doc in item_reviews_list]
    
    # retrieve user and item topics vectors
    user_vecs = get_topic_vectors(lda, user_corpus)
    item_vecs = get_topic_vectors(lda, item_corpus)
    
    # generate a mapping 
    user_idx_map = {k: unique_users[k] for k in range(len(unique_users))}
    item_idx_map = {k: unique_items[k] for k in range(len(unique_items))}
    user_vec_map = {k: v for k, v in zip(unique_users, user_vecs)}
    item_vec_map = {k: v for k, v in zip(unique_items, item_vecs)}
    
    # loading user topic vectors into DF
    user_vecs = pd.DataFrame.from_dict(user_vec_map, orient='index')
    user_vecs.index.name = 'reviewerID'
    # loading item topic vectors into DF
    item_vecs = pd.DataFrame.from_dict(item_vec_map, orient='index')
    item_vecs.index.name = 'asin'
    
    return user_idx_map, user_vecs, item_idx_map, item_vecs

In [9]:
user_idx_map, user_vecs, item_idx_map, item_vecs = generate_user_item_vectors(lda_model, train)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4729/4729 [00:04<00:00, 975.66it/s]


In [10]:
# converting factors into numpy obj
user_factors = user_vecs.to_numpy()
item_factors = item_vecs.to_numpy()

In [11]:
# check user factors
user_factors[0,:]

array([0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.14620695, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.00133466, 0.09169909, 0.00133466, 0.57021385, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.13048595, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466,
       0.00133466, 0.00133466, 0.00133466, 0.00133466, 0.00133466],
      dtype=float32)

In [12]:
# check item factors
item_factors[0,:]

array([4.4094846e-05, 4.4094846e-05, 4.8933003e-02, 7.1832053e-02,
       8.0743708e-02, 4.4094846e-05, 4.4094846e-05, 4.4094846e-05,
       1.0947618e-01, 4.4094846e-05, 3.5948649e-02, 4.4094846e-05,
       4.4094846e-05, 1.8174406e-02, 4.4094846e-05, 2.4109175e-02,
       4.4094846e-05, 4.2666155e-03, 1.6192118e-02, 4.4094846e-05,
       1.1335680e-02, 4.4094846e-05, 4.4094846e-05, 4.4094846e-05,
       4.4094846e-05, 4.4094846e-05, 4.4094846e-05, 4.4094846e-05,
       4.4094846e-05, 4.4094846e-05, 4.4094846e-05, 4.4094846e-05,
       4.4094846e-05, 4.4094846e-05, 4.8612170e-02, 3.6693811e-01,
       4.4094846e-05, 4.4094846e-05, 4.4094846e-05, 4.4094846e-05,
       4.4094846e-05, 7.4354723e-02, 4.4094846e-05, 4.4094846e-05,
       1.8433521e-02, 4.4094846e-05, 5.2157536e-02, 1.0093035e-02,
       4.4094846e-05, 6.9441656e-03], dtype=float32)

# Utility Functions

In [13]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in tqdm(predictions):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in tqdm(top_n.items()):
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def recall_at_k(asins, predicted_asins, k=10):
    # number of relevant items
    set_actual = set(asins)
    set_preds = set(predicted_asins)
    num_relevant = len(set_actual.intersection(set_preds))
    
    # calculating recall@K - relevant / total relevant items
    recall_at_k = num_relevant / len(asins)
    
    return recall_at_k

def novelty_at_k(item_popularity, predicted_asins, k=10):
    """
    """
    # finding avg novelty
    popularity_sum = item_popularity.loc[predicted_asins].sum()
    novelty_at_k = ((k*1) - popularity_sum) / k
    
    return novelty_at_k

def generate_item_popularity(train: pd.DataFrame) -> pd.DataFrame:
    """
    """
    
    # create a mapping of item popularatity
    # based on sum(item's review / max reviews) / no items
    max_reviews = (train.groupby(['asin'])
                   .agg({'processedReviewText': 'count'})
                   .max()
                   .values[0])
    item_popularity = (train.groupby(['asin'])
                       .agg({'processedReviewText': 'count'})
                       .apply(lambda x: x/max_reviews))
    
    return item_popularity
    

def evaluate_recommendations(top_ns: dict, user_rating_history: pd.DataFrame, item_popularity: pd.DataFrame, k=10) -> pd.DataFrame:
    """
    
    Args:
        top_ns
        user_rating_history
    """
    
    test_recommendations = pd.DataFrame(top_ns.items(), columns=["reviewerID", "pred_asin"])
    test_recommendations['pred_asin'] = test_recommendations['pred_asin'].apply(lambda x: [i[0] for i in x])
    
    # combined test history and recommendations
    test_merged = pd.merge(user_rating_history, test_recommendations, on="reviewerID", how="inner")
    
    # generating recall@k metrics
    test_merged["recall@k"] = test_merged.apply(lambda x: recall_at_k(x.asin, x.pred_asin, k=k), axis=1)
    test_merged["novelty@k"] = test_merged.apply(lambda x: novelty_at_k(item_popularity, x.pred_asin, k=k), axis=1)
    average_recall_at_k = test_merged["recall@k"].mean()
    average_novelty_at_k = test_merged["novelty@k"].mean()
    
    print(f"The TI-MF has an average recall@{k}: {average_recall_at_k:.5f}, average novelty@{k}: {average_novelty_at_k:.5f}")
    
    return test_merged

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [14]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [15]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,2,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1TCSC0YWT82Q0,5.0,I love ethnic foods and to cook them. I recent...,2013-08-03,love ethnic food cook recently purchase produc...
1,8,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1Z7Y2GMAP9SRY,5.0,I like to make my own curry but this is a tast...,2014-06-27,like curry tasty alternative use base kind dif...
2,23,B00004S1C5,"Ateco Food Coloring Kit, 6 colors","['Grocery & Gourmet Food', 'Cooking & Baking',...",A14YSMLYLJEMET,1.0,This product is no where near natural / organi...,2013-03-29,product near natural organic wish review purch...
3,31,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",A2F488C4PLWGEI,5.0,If my wife drinks a cup of this tea when she f...,2014-03-23,wife drink cup tea feel attack come help avoid...
4,32,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AO1HXV7DWZZIR,5.0,I don't know about the medicinal aspects of th...,2014-02-06,know medicinal aspect tea flavor downright scr...
28001,77519,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A1WT3TVHANP7ZF,3.0,Hmmm. I really wanted to love this sweetener. ...,2014-07-22,hmmm want love sweetener half sugar half stevi...
28002,77520,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A3NEAETOSXDBOM,5.0,"I confess I have a sweet tooth, and love the t...",2014-06-30,confess sweet tooth love taste sugar recognize...
28003,77521,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",AD1ZOPB0BBEHB,4.0,"It has a little of the stevia aftertaste, but ...",2014-07-17,little stevia aftertaste fair compromise able ...
28004,77522,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A18ECVX2RJ7HUE,5.0,i love marinade for grilled flank steak or lon...,2014-05-30,love marinade grilled flank steak london broil...
28005,77523,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2G04D4QZAXL15,3.0,I've been using Truvia (a form of stevia) on m...,2014-05-27,use truvia form stevia cereal greek yogurt yea...


In [16]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [17]:
print(test_user_history)

                  reviewerID  \
0      A00177463W0XWB16A9O05   
1      A022899328A0QROR32DCT   
2      A068255029AHTHDXZURNU   
3      A06944662TFWOKKV4GJKX   
4             A1004703RC79J9   
...                      ...   
13274          AZWRZZAMX90VT   
13275          AZXKAH2DE6C8A   
13276          AZXON596A1VXC   
13277          AZYXC63SS008M   
13278          AZZ5ASC403N74   

                                                    asin  
0                               [B00474OR8G, B00BFM6OAW]  
1                                           [B00CMQDKES]  
2                                           [B001FA1K2G]  
3                                           [B000GFYRHG]  
4                                           [B003GTR8IO]  
...                                                  ...  
13274  [B0007R9L4M, B000CN7BMA, B001EQ5D1K, B002VT3GX...  
13275   [B000MAK41I, B004X8TJP2, B006H34CUS, B007W14RMM]  
13276                           [B001EO5S0I, B00271QQ7Q]  
13277                    

# Preparing Dataset for Surprise's Algorithm

In [18]:
# create reader
reader = Reader(rating_scale=(1,5))
# generate data required for surprise
data = Dataset.load_from_df(train[["reviewerID", "asin", "overall"]], reader)
# generating trainset
trainset = data.build_full_trainset()

# Instantiate Pre-Initialised Matrix Factorization (Topic Modelling)

In [19]:
# instantiating ti_mf
ti_mf = cf.PreInitialisedMF(user_map=user_idx_map,
                            item_map=item_idx_map,
                            user_factor=user_factors,
                            item_factor=item_factors,
                            learning_rate=LR_ALL,
                            beta=BETA,
                            num_epochs=N_EPOCHS,
                            num_factors=50)

In [20]:
%%time
# fitting to training data
ti_mf.fit(trainset, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
CPU times: user 3min 59s, sys: 866 ms, total: 4min
Wall time: 4min 1s


In [21]:
%%time
# generate candidate items for user to predict rating
testset = trainset.build_anti_testset()

CPU times: user 32.1 s, sys: 1.58 s, total: 33.7 s
Wall time: 34 s


In [22]:
%%time
# predict ratings for all pairs (u, i) that are NOT in the training set
candidate_items = ti_mf.test(testset, verbose=False)

CPU times: user 7min 45s, sys: 1min 19s, total: 9min 5s
Wall time: 9min 21s


## Loop through N = {10, 25, 30, 45}

In [23]:
# generate item popularity
item_popularity = generate_item_popularity(train)

In [24]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    top_ns = get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = evaluate_recommendations(top_ns, test_user_history, item_popularity, n)
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13397/13397 [00:39<00:00, 337.55it/s]


The TI-MF has an average recall@10: 0.01268, average novelty@10: 0.90050


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13397/13397 [00:40<00:00, 327.36it/s]


The TI-MF has an average recall@25: 0.01994, average novelty@25: 0.92528


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13397/13397 [00:41<00:00, 323.50it/s]


The TI-MF has an average recall@30: 0.02180, average novelty@30: 0.93025


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13397/13397 [00:38<00:00, 349.54it/s]


The TI-MF has an average recall@45: 0.02647, average novelty@45: 0.93792


# Evaluate N-Recommendations

In [25]:
def retrieve_recommendations(train: pd.DataFrame, top_ns: dict):
    """
    """
    # generating a random user
    random_user = np.random.choice(list(train['reviewerID'].unique()), 1)[0]
    print(f"For user: {random_user}:")
    print(f"Purchase History:\n{train[train['reviewerID'] == random_user][['asin', 'title']]}")

    # find the recommendations
    print(f"\nRecommending:\n")
    recommendations = (train[train['asin']
                             .isin([i[0] for i in top_ns[random_user]])][['asin', 'title']]
                       .drop_duplicates(subset='asin')
                       .set_index('asin'))
    print(f"{recommendations.loc[[i[0] for i in top_ns[random_user]]].reset_index()}")

## N=10

In [31]:
top_ns_10 = n_recommendations[10][0]
retrieve_recommendations(train, top_ns_10)

For user: A26NSWOAKB86LM:
Purchase History:
             asin                                              title
34174  B00473SRCY  Kona Coast Macadamia Nut Pancake Mix, 24-Ounce...

Recommending:

         asin                                              title
0  B000EDBPO8  Bob's Red Mill White Rice Flour, Organic, 24-O...
1  B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
2  B000G82L62  Lundberg Family Farms Wild Blend Rice, 16 Ounc...
3  B001PEWJWC  Garbanzo Beans aka Chickpeas or Ceci Beans | N...
4  B00DS842HS  Viva Naturals Organic Extra Virgin Coconut Oil...
5  B0001M0Z6Q  Spicy World Peppercorn (Whole)-Black Tellicher...
6  B00014JNI0  YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...
7  B003OGKCDC  Nature's Way Organic Extra Virgin Coconut Oil-...
8  B000F4D5GC  Let's Do Organic Shredded, Unsweetened Coconut...
9  B00338DSQ4      Barilla Spaghetti Pasta, 32 Ounce (Pack of 6)


## N=25

In [27]:
top_ns_25 = n_recommendations[25][0]
retrieve_recommendations(train, top_ns_25)

For user: A28703KGCS8P2P:
Purchase History:
             asin                                              title
31250  B003AYEHIO  Starwest Botanicals Organic Dried Lavender Flo...

Recommending:

          asin                                              title
0   B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
1   B00DS842HS  Viva Naturals Organic Extra Virgin Coconut Oil...
2   B000EDBPO8  Bob's Red Mill White Rice Flour, Organic, 24-O...
3   B0001M0Z6Q  Spicy World Peppercorn (Whole)-Black Tellicher...
4   B001PEWJWC  Garbanzo Beans aka Chickpeas or Ceci Beans | N...
5   B000G82L62  Lundberg Family Farms Wild Blend Rice, 16 Ounc...
6   B003OGKCDC  Nature's Way Organic Extra Virgin Coconut Oil-...
7   B00014JNI0  YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...
8   B000F4D5GC  Let's Do Organic Shredded, Unsweetened Coconut...
9   B000JMAXMY                   Mustard Seeds 7oz by Spicy World
10  B000Z93FQC               Y.S. Eco Bee Farms Raw Honey - 22 oz
11  B000HD

## N=30

In [28]:
top_ns_30 = n_recommendations[30][0]
retrieve_recommendations(train, top_ns_30)

For user: A31SXZGGA9AGN2:
Purchase History:
             asin                                              title
42595  B006307HT8  Chef Paul Prudhomme's Magic Seasoning Blends N...
46333  B00A64NLOM  WERTHER'S ORIGINAL Creamy Caramel Filled Hard ...

Recommending:

          asin                                              title
0   B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
1   B00DS842HS  Viva Naturals Organic Extra Virgin Coconut Oil...
2   B000EDBPO8  Bob's Red Mill White Rice Flour, Organic, 24-O...
3   B0001M0Z6Q  Spicy World Peppercorn (Whole)-Black Tellicher...
4   B001PEWJWC  Garbanzo Beans aka Chickpeas or Ceci Beans | N...
5   B003OGKCDC  Nature's Way Organic Extra Virgin Coconut Oil-...
6   B00014JNI0  YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...
7   B000G82L62  Lundberg Family Farms Wild Blend Rice, 16 Ounc...
8   B000F4D5GC  Let's Do Organic Shredded, Unsweetened Coconut...
9   B000HDK0DC  YumEarth Organic Lollipops, Assorted Flavors, ...
10  B00

## N=45

In [29]:
top_ns_45 = n_recommendations[45][0]
retrieve_recommendations(train, top_ns_45)

For user: A24XFXUCAMWP0D:
Purchase History:
             asin                                              title
4834   B000EDG3UE  Bob's Red Mill Organic Grain Quinoa, 26 Ounce ...
47038  B00BSD9C5M          Healthworks Goji Berries Raw Organic, 2lb

Recommending:

          asin                                              title
0   B0001M0Z6Q  Spicy World Peppercorn (Whole)-Black Tellicher...
1   B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
2   B001PEWJWC  Garbanzo Beans aka Chickpeas or Ceci Beans | N...
3   B00DS842HS  Viva Naturals Organic Extra Virgin Coconut Oil...
4   B003OGKCDC  Nature's Way Organic Extra Virgin Coconut Oil-...
5   B000EDBPO8  Bob's Red Mill White Rice Flour, Organic, 24-O...
6   B000F4D5GC  Let's Do Organic Shredded, Unsweetened Coconut...
7   B000G82L62  Lundberg Family Farms Wild Blend Rice, 16 Ounc...
8   B000Z93FQC               Y.S. Eco Bee Farms Raw Honey - 22 oz
9   B000JMAXMY                   Mustard Seeds 7oz by Spicy World
10  B00